In [48]:
import sqlite3
import math
import os
import pandas as pd
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from graphviz import Source
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output


In [28]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/FPA_FOD_20170508.sqlite")
df = pd.read_sql_query("SELECT datetime(DISCOVERY_DATE) as DISCOVERY_DATE, datetime(CONT_DATE) as CONT_DATE,STATE,COUNTY,LONGITUDE,LATITUDE,STAT_CAUSE_CODE,FIRE_SIZE, FIRE_SIZE_CLASS  from Fires", con)

In [29]:
#Function for calculating remainder of the new data compa
def remaining_data(olddf,newdf):
    x = len(df)
    print(us_fires.shape)

    return ('The new dataframe is',
        round(len(us_fires)/x * 100, 3), 
        '% of the original dataset')

In [30]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'])
df['DAYS_TO_EXT'] = df['CONT_DATE'] - df['DISCOVERY_DATE']
df['DAYS_TO_EXT'] = df['DAYS_TO_EXT'].dt.days


In [31]:
df = df.dropna()

In [44]:
#Save this for training later
us_fires = df[['LONGITUDE','LATITUDE','DAYS_TO_EXT','FIRE_SIZE']]
us_fires['LONGITUDE'] = us_fires['LONGITUDE'].astype(int)
us_fires['LATITUDE'] = us_fires['LATITUDE'].astype(int)
us_fires['DAYS_TO_EXT'] = us_fires['DAYS_TO_EXT'].astype(int)
us_fires['FIRE_SIZE'] = us_fires['FIRE_SIZE'].astype(int)
size_class = us_fires['FIRE_SIZE']
us_fires.dtypes

C:\Users\Johan\AppData\Local\Temp/ipykernel_4876/701159789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_fires['LONGITUDE'] = us_fires['LONGITUDE'].astype(int)
C:\Users\Johan\AppData\Local\Temp/ipykernel_4876/701159789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_fires['LATITUDE'] = us_fires['LATITUDE'].astype(int)
C:\Users\Johan\AppData\Local\Temp/ipykernel_4876/701159789.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

LONGITUDE      int32
LATITUDE       int32
DAYS_TO_EXT    int32
FIRE_SIZE      int32
dtype: object

In [45]:
X_train, X_test, y_train, y_test = train_test_split(us_fires, size_class, test_size=0.25)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))

Our training data has 502240 rows
Our test data has 167414 rows


In [46]:
classifier = DecisionTreeClassifier(max_depth=3)
classifier.fit(X_train.values, y_train.values)

DecisionTreeClassifier(max_depth=3)

In [47]:
sample = X_test.head(10)
sample['FIRE_SIZE'] = classifier.predict(sample)
sample

C:\Users\Johan\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\Johan\AppData\Local\Temp/ipykernel_4876/4163555083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['FIRE_SIZE'] = classifier.predict(sample)


,LONGITUDE,LATITUDE,DAYS_TO_EXT,FIRE_SIZE
1148642,-87,33,0,3
1577926,-115,40,0,0
38122,-121,43,0,0
1510596,-80,35,0,0
1686770,-83,32,0,0
698304,-88,30,0,3
1266475,-83,36,0,3
1871936,-122,40,0,0
143012,-89,30,11,3
1153371,-86,33,0,0


In [49]:
tree_plot = Source(tree.export_graphviz(classifier, out_file=None, 
                            feature_names=X_train.columns, class_names=['Dead', 'Alive'], 
                            filled=True, rounded=True, special_characters=True))
tree_plot

IndexError: list index out of range